# Uber Prize Starter Kit Python Utilities: Tutorial

To simplify interaction with the Docker-based simulation execution and evaluation, we've provided a set of Python utilities (located in the `/utilities` folder of the Starter-Kit repository). This notebook demonstrates how they may be used to accomplish the following tasks:

 - Starting a simulation or several simulations
 - Checking simulation completion
 - Retrieving the simulation score in a convenient Pandas `DataFrame` format.
 - Generating fake data 

*Note*: It is assumed that this notebook is started from the `/examples` folder.

In [1]:

# Adding the module to the path for future import
import sys
import os
import docker
from pathlib import Path
# Note that the following is idempotent when this notebook is run from "/examples"
os.chdir('../utilities')
%load_ext autoreload
%autoreload 2

## The `competition_executor` Module

A `CompetitionContainerExecutor` object may be used to start, stop, and gather information about containers running simulations and/or completed simulation scores and stats.

In [2]:
from competition_executor import CompetitionContainerExecutor

path_input = (Path(Path.cwd()).parent / "submission-inputs").absolute()
path_ouput = (Path(Path.cwd()).parent / "output").absolute()

my_executor = CompetitionContainerExecutor(input_root=path_input,
                                  output_root=path_ouput)

# Note: Instantiating a CompetitionContainerExecutor with path arguments is not strictly necessary. Each simulation
# can be run with its own set of input and/or output arguments. However, if you prefer to designate
# a single directory for inputs and/or a single directory for outputs, then, for convenience, 
# you may pass those arguments in here and avoid re-entering them for each simulation run.

Run the simulation using the `run_sumulation` method. For example:

In [3]:
# Note: the `submission_id`s for each run must be unique; however, this cell may be run idempotently
try:
    my_submission = my_executor.client.containers.get('my_submission')
    my_submission.stop()
    my_submission.remove()
    print("Found container from previous run, removing and creating new sim container...")
except docker.errors.NotFound:
    print("Creating new simulation container...")
    
my_executor.run_simulation('my_submission', num_iterations=2, num_cpus=4)

# Note that a dictionary indicating which containers IDs already exist can be accessed.
for key, value in my_executor.containers.items():
    print("Container ID : {0}".format(key))

Creating new simulation container...
Container ID : my_submission


You may view the **logs** of a specific simulation as follows:

In [4]:
logs = my_executor.output_simulation_logs('my_submission')

04:06:22.049 DEBUG beam.utils.BeamConfigUtils$ - Loading beam config from fixed-data/siouxfalls/siouxfalls-1k.conf.
04:06:24.407 DEBUG b.sim.config.MatSimBeamConfigBuilder - Couldn't instantiate MatsimParameters  'org.matsim.run.OldToNewPlanCalcScoreConfigGroup.ModeParams'. It doesn't have default public constructor.Falling back to setAccessible(). Cause : Class beam.sim.config.MatSimBeamConfigBuilder$$anonfun$1 can not access a member of class org.matsim.run.OldToNewPlanCalcScoreConfigGroup$ModeParams with modifiers ""
04:06:24.492 DEBUG b.sim.config.MatSimBeamConfigBuilder - Couldn't instantiate MatsimParameters  'org.matsim.core.config.groups.PlanCalcScoreConfigGroup.ModeParams'. It doesn't have default public constructor.Falling back to setAccessible(). Cause : Class beam.sim.config.MatSimBeamConfigBuilder$$anonfun$1 can not access a member of class org.matsim.core.config.groups.PlanCalcScoreConfigGroup$ModeParams with modifiers ""
04:06:24.827 DEBUG beam.utils.FileUtils$ - Beam ou

Submissions run for a certain amount of time. If you interrupt it before the end, you will not get any outputs. You can **check if a submission is finished** with the following method:

In [5]:
my_executor.check_if_submission_complete('my_submission')

False

When a simulation run is done, you can import its **results**:

In [6]:
scores, stats = my_executor.get_submission_scores_and_stats('my_submission')

NameError: Simulation my_submission is still running.

The **scores** and **statistics** are stored in pandas DataFrames which contains the information described [here](https://github.com/vgolfier/Uber-Prize-Starter-Kit/blob/master/docs/Understanding_the_outputs_and_the%20scoring_function.md).

At the end of any run and to avoid any conflicts between submission ids (particularly if you want to reuse names), it is advised to **clean up containers** calling the following method:

In [7]:
my_executor.stop_all_simulations()

## The `input_sampler` module:

Randomly sampled data may be used to either initialize the re-planning algorithm or otherwise test the simulation. The `input_sampler` has been provided towards this end. This subsection provides an example of how synthetic random input files may be generated for each of the available input policies to the simulation.

In [33]:
from input_sampler import *
# Specify the common string for the scenario name 
# (currently only "siouxfalls", which refers to the Sioux Falls scenario).
SCENARIO_NAME = 'sioux_faux'
agency = 'sioux_faux_bus_lines'

# Set the paths appropriately
DIR = Path(__name__).absolute().parent.parent
sys.path.append(str(DIR))
DATA_DIR = DIR / 'reference-data/'

agency_dict = scenario_agencies(DATA_DIR,SCENARIO_NAME)
    # Create a lazy cache of GTFS data for the agency:
sf_gtfs_manager = AgencyGtfsDataManager(agency_dict[agency])

# Sample each input five (5) times (naming of functions is indicative of type of input sampled).
freq_df = sample_frequency_adjustment_input(5, sf_gtfs_manager)
mode_subsidy_df = sample_mode_subsidies_input(5)
vehicle_fleet_mix_df = sample_vehicle_fleet_mix_input(5, sf_gtfs_manager)

# Generated inputs may now be saved in, say, /submission-inputs or some other directory. 
# Remember the location of this directory when executing new simulations.

In [34]:
vehicle_fleet_mix_df

,routeId,agencyId,vehicleType
0,1351,217,CAR-TYPE-DEFAULT
1,1343,217,CAR-TYPE-DEFAULT
2,1344,217,BUS-STD-ART
3,1341,217,BUS-SMALL-HD
4,1345,217,BUS-STD-HD
